In [1]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer
from sklearn.utils import shuffle

from tools import plot_confusion_matrix

In [2]:
df = pd.read_csv('../data/datatran2017.csv', sep=';', encoding="latin1")
df.shape

(89518, 30)

In [5]:
df.classificacao_acidente.value_counts()

Com Vítimas Feridas    53647
Sem Vítimas            30689
Com Vítimas Fatais      5182
Name: classificacao_acidente, dtype: int64

In [3]:
df1 = pd.read_csv('../data/datatran2016_atual.csv', sep=';', encoding="latin1")
df1.shape

(96363, 25)

In [13]:
def fix_date(date):
    l = date.split('/')
    return '20{}-{}-{}'.format(l[2], l[1], l[0])
    

data = df1.data_inversa.apply(lambda x: fix_date(x))
df1.data_inversa = data

In [14]:
df1.classificacao_acidente.value_counts()

Com Vítimas Feridas    54873
Sem Vítimas            34478
Com Vítimas Fatais      5355
Ignorado                1656
Name: classificacao_acidente, dtype: int64

In [15]:
df_completo = pd.concat([df, df1])

In [17]:
df_completo.tail()

,br,causa_acidente,classificacao_acidente,condicao_metereologica,data_inversa,delegacia,dia_semana,fase_dia,feridos,feridos_graves,...,pessoas,regional,sentido_via,tipo_acidente,tipo_pista,tracado_via,uf,uop,uso_solo,veiculos
96358,163.0,Falta de atenção,Com Vítimas Fatais,Ceu Claro,2016-11-10,NaN,Quinta,Pleno dia,1,0,...,2,NaN,Crescente,Saída de Pista,Simples,Reta,MT,NaN,Urbano,1
96359,428.0,Ingestão de álcool,Com Vítimas Feridas,Ceu Claro,2016-01-01,NaN,Sexta,Plena noite,1,1,...,3,NaN,Decrescente,Colisão traseira,Simples,Reta,PE,NaN,Urbano,2
96360,267.0,Velocidade incompatível,Com Vítimas Feridas,Chuva,2016-12-10,NaN,Sábado,Pleno dia,1,0,...,1,NaN,Decrescente,Saída de Pista,Simples,Curva,MG,NaN,Rural,1
96361,393.0,Defeito mecânico em veículo,Sem Vítimas,Ceu Claro,2016-12-28,NaN,Quarta,Pleno dia,0,0,...,3,NaN,Crescente,Colisão lateral,Simples,Reta,RJ,NaN,Urbano,2
96362,40.0,Outras,Com Vítimas Feridas,Chuva,2016-12-04,NaN,Domingo,Anoitecer,5,2,...,13,NaN,Crescente,Capotamento,Dupla,Curva,RJ,NaN,Rural,1


In [19]:
df_completo.classificacao_acidente.value_counts()

Com Vítimas Feridas    108520
Sem Vítimas             65167
Com Vítimas Fatais      10537
Ignorado                 1656
Name: classificacao_acidente, dtype: int64

In [21]:
df = df_completo[df_completo.classificacao_acidente!='Ignorado']
df.shape

(184225, 30)

In [22]:
df['hora'] = df.horario.apply(lambda x: x.split(':')[0])
df['ano'] = df.data_inversa.apply(lambda x: x.split('-')[0])
df['mes'] = df.data_inversa.apply(lambda x: x.split('-')[1])
df['dia'] = df.data_inversa.apply(lambda x: x.split('-')[2])

df.drop(['horario', 'data_inversa', 'id'], axis=1, inplace=True)

/Users/leportella/.virtualenvs/udacity/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/leportella/.virtualenvs/udacity/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/leportella/.virtualenvs/udacity/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [23]:
value = {
    'Com Vítimas Feridas': 1,
    'Sem Vítimas': 0,
    'Com Vítimas Fatais': 2
}

In [25]:
df = df[~df.classificacao_acidente.isnull()]
df.shape

(184224, 31)

In [29]:
df['target'] = df.classificacao_acidente.apply(lambda x: value[x])
df.shape

(184224, 32)

In [30]:
df.to_csv('../data/datatran_2016-2017.csv', index=False)